In [ ]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os
import time

ak.connect("n33", 5555)
p = 0
num_nodes = 100_000

start = time.time()
temp_prop_graph = ar.gnp(num_nodes, p, create_using=ar.PropGraph)
end = time.time()
build_time = end - start

print(f"Building property graph with {len(temp_prop_graph)} vertices and "
        f"{temp_prop_graph.size()} "
        f"edges took {round(build_time,2)} seconds.")

### Generate node labels and edge relationships for the graph.
# 1. Extract node and edge information.
num_edges = temp_prop_graph.size()
num_nodes = len(temp_prop_graph)
edges = temp_prop_graph.edges()
nodes = temp_prop_graph.nodes()

##############################################
# NEW SECTION: Generate random node and edge attributes
##############################################

# For nodes:
# 2.1. Categorical from ["blue", "red", "yellow"]
# colors = ak.array(["blue", "red"])
# node_colors = colors[ak.randint(0, len(colors), num_nodes)]

# 2.2. Integer from 10 to 12 (inclusive)
node_ints = ak.randint(10, 12, num_nodes)

# 2.3. Boolean from [False, True]
node_bools = ak.randint(0, 1, num_nodes, dtype=ak.bool)

# For edges:
# 2.4. Randomly choose from [5, 10]
edge_ints = ak.randint(10, 12, num_edges)

##############################################
# Create dataframes with the new attributes
##############################################

edge_df = ak.DataFrame({
    "src": edges[0],
    "dst": edges[1],
    "edge_attr": edge_ints
})

node_df = ak.DataFrame({
    "nodes": nodes,
    # "color": node_colors,
    "int_attr": node_ints,
    "bool_attr": node_bools
})

##############################################
# Create the new property graph with these attributes
##############################################

prop_graph = ar.PropGraph()
prop_graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst")
prop_graph.load_node_attributes(node_df, node_column="nodes")
print("Property graph created with random node and edge attributes.")



### Create the subgraph we are searching for.
# 1. Create labels and relationships to search for.
# src_list = [0, 1, 2, 1]
# dst_list = [1, 2, 0, 3]

src_list = [2, 3, 1, 3]
dst_list = [3, 1, 2, 0]
src_subgraph = ak.array(src_list)
dst_subgraph = ak.array(dst_list)

subgraph_nodes = list(set(src_list) | set(dst_list))
subgraph_nodes.sort()

# labels1_subgraph = ak.array(["blue", "blue", "red", "red"])
labels2_subgraph = ak.array([10, 12, 10, 12])
labels3_subgraph = ak.array([True, True, True, True])

rels1_subgraph = ak.array([10, 12, 10, 12])

connected to arkouda server tcp://*:5555


Building property graph with 50000 vertices and 1250667 edges took 7.24 seconds.
Property graph created with random node and edge attributes.


In [ ]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os
import time

ak.connect("n33", 5555)
p = 0.0005
num_nodes = 50_000

# Generate graph using NetworkX
graph = nx.gnp_random_graph(num_nodes, p, directed=True, seed=42)
src_list, dst_list = zip(*graph.edges())
src_ak = ak.array(src_list)
dst_ak = ak.array(dst_list)

start = time.time()
temp_prop_graph = ar.PropGraph()
end = time.time()
build_time = end - start
print("networkx graph generation....")
print(f"Building property graph with {num_nodes} vertices and "
      f"{len(src_list)} edges took {round(build_time,2)} seconds.")

### Generate node labels and edge relationships for the graph.
# 1. Extract node and edge information.
num_edges = len(src_list)
edges = (src_ak, dst_ak)
nodes = ak.arange(num_nodes)

##############################################
# NEW SECTION: Generate random node and edge attributes
##############################################

# For nodes:
# 2.1. Categorical from ["blue", "red", "yellow"]
colors = ak.array(["blue", "red", "yellow"])
node_colors = colors[ak.randint(0, len(colors), num_nodes)]

# 2.2. Integer from 10 to 12 (inclusive)
node_ints = ak.randint(10, 13, num_nodes)

# 2.3. Boolean from [False, True]
node_bools = ak.randint(0, 2, num_nodes, dtype=ak.bool)

# For edges:
# 2.4. Randomly choose from [5, 10]
edge_ints = ak.randint(5, 11, num_edges)

##############################################
# Create dataframes with the new attributes
##############################################

edge_df = ak.DataFrame({
    "src": edges[0],
    "dst": edges[1],
    "edge_attr": edge_ints
})

node_df = ak.DataFrame({
    "nodes": nodes,
    "color": node_colors,
    "int_attr": node_ints,
    "bool_attr": node_bools
})

##############################################
# Create the new property graph with these attributes
##############################################

prop_graph = ar.PropGraph()
prop_graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst")
prop_graph.load_node_attributes(node_df, node_column="nodes")
print("Property graph created with random node and edge attributes.")

### Create the subgraph we are searching for.
# 1. Create labels and relationships to search for.
src_list = [2, 3, 1, 3]
dst_list = [3, 1, 2, 0]
src_subgraph = ak.array(src_list)
dst_subgraph = ak.array(dst_list)

subgraph_nodes = list(set(src_list) | set(dst_list))
subgraph_nodes.sort()

labels1_subgraph = ak.array(["blue", "blue", "red", "red"])
labels2_subgraph = ak.array([10, 12, 10, 12])
labels3_subgraph = ak.array([True, True, True, True])

rels1_subgraph = ak.array([10, 12, 10, 12])


connected to arkouda server tcp://*:5555


In [9]:
"""VF2-SI STRUCTURAL TRUE"""
edge_df_h = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = None, algorithm_type = "si", 
                                           reorder = True, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

We found 388559.0 monos inside of the graph


In [13]:
"""VF2-SI STRUCTURAL FALSE"""
edge_df_h = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = None, algorithm_type = "si", 
                                           reorder = False, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

We found 388559.0 monos inside of the graph


In [ ]:
"""VF2-PS STRUCTURAL TRUE"""
edge_df_h = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = None, algorithm_type = "ps", 
                                           reorder = True, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [ ]:
"""VF2-PS STRUCTURAL FALSE"""
edge_df_h = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = None, algorithm_type = "ps", 
                                           reorder = False, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [ ]:
"""VF2-SI VERTEX PROBABILITIES OR"""
edge_df_h = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), 
                          # "lbls1": labels1_subgraph, 
                          "lbls2": labels2_subgraph,
                          "lbls3": labels3_subgraph
})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
sg.load_node_attributes(node_df_h, node_column="nodes")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "or", algorithm_type = "si", 
                                           reorder = False, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [ ]:
"""VF2-SI VERTEX PROBABILITIES AND"""
edge_df_h = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), 
                          # "lbls1": labels1_subgraph, 
                          "lbls2": labels2_subgraph,
                          "lbls3": labels3_subgraph
})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
sg.load_node_attributes(node_df_h, node_column="nodes")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "and", algorithm_type = "si", 
                                           reorder = False, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [ ]:
"""VF2-SI VERTEX AND EDGE PROBABILITIES OR"""
edge_df_h = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph, 
                          "rels1": rels1_subgraph
})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), 
                          # "lbls1": labels1_subgraph, 
                          "lbls2": labels2_subgraph,
                          "lbls3": labels3_subgraph
})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
sg.load_node_attributes(node_df_h, node_column="nodes")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "or", algorithm_type = "si", 
                                           reorder = False, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [ ]:
"""VF2-SI VERTEX AND EDGE PROBABILITIES AND"""
edge_df_h = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph, 
                          "rels1": rels1_subgraph
})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), 
                          # "lbls1": labels1_subgraph, 
                          "lbls2": labels2_subgraph,
                          "lbls3": labels3_subgraph
})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
sg.load_node_attributes(node_df_h, node_column="nodes")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "or", algorithm_type = "si", 
                                           reorder = False, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [ ]:
"""VF2-PS VERTEX PROBABILITIES TRUE"""
edge_df_h = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), 
                          # "lbls1": labels1_subgraph, 
                          "lbls2": labels2_subgraph,
                          "lbls3": labels3_subgraph
})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
sg.load_node_attributes(node_df_h, node_column="nodes")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "or", algorithm_type = "ps", 
                                           reorder = False, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [ ]:
"""VF2-PS VERTEX AND EDGE PROBABILITIES"""
edge_df_h = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph, 
                          "rels1": rels1_subgraph
})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), 
                          # "lbls1": labels1_subgraph, 
                          "lbls2": labels2_subgraph,
                          "lbls3": labels3_subgraph
})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
sg.load_node_attributes(node_df_h, node_column="nodes")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "or", algorithm_type = "ps", 
                                           reorder = False, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")